In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

from PIL import Image
import cv2
import albumentations as A

import random
import time
import os
from tqdm.auto import tqdm

# !pip install -q segmentation-models-pytorch
# !pip install -q torchsummary

from torchsummary import summary
import segmentation_models_pytorch as smp
import rasterio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.UnetPlusPlus('efficientnet-b0', encoder_weights='imagenet', classes=1, activation=None, in_channels=3)

In [ ]:
model.load_state_dict(torch.load('Best_Dice.pt'))
model.to(device)

mean = [0.154, 0.200, 0.154]
std = [0.061, 0.072, 0.058]

In [ ]:
def model_predict(img_path):
    # 이미지 로드 및 전처리
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))  # 크기 일치

    t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    image = t(img)
    image = image.to(device)

    model.eval()

    with torch.no_grad():
        image = image.unsqueeze(0)
        output = model(image)
        output = torch.sigmoid(output)

        threshold = 0.000001  # binary classification에 대한 일반적인 임계값
        binary_mask = (output > threshold).float()
        pred_mask = binary_mask.cpu().squeeze(0).squeeze(0).numpy()

    # 마스크를 uint8로 변환
    seg = (pred_mask * 255).astype(np.uint8)

    # 메인 이미지 준비
    main = img.copy()  # RGB 이미지 유지

    # 외부 윤곽선 찾기
    contours, _ = cv2.findContours(seg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # 윤곽선 그리기 (빨간색으로 단일 클래스 표시)
    cv2.drawContours(main, contours, -1, (255, 0, 0), 4)  # 빨간색, 두께 2

    return img, pred_mask, main

In [ ]:
img, pred_mask, main = model_predict('../test_2.jpg')

# 결과 표시
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('SafeZone From Fire :: Input')
ax1.axis('off')

ax2.imshow(pred_mask, cmap='gray')
ax2.set_title('SafeZone From Fire :: Mask')
ax2.axis('off')

ax3.imshow(main)
ax3.set_title('SafeZone From Fire :: Contours')
ax3.axis('off')

plt.show()

In [ ]:
img, pred_mask, main = model_predict('../test_1.jpg')

# 결과 표시
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('SafeZone From Fire :: Input')
ax1.axis('off')

ax2.imshow(pred_mask, cmap='gray')
ax2.set_title('SafeZone From Fire :: Mask')
ax2.axis('off')

ax3.imshow(main)
ax3.set_title('SafeZone From Fire :: Contours')
ax3.axis('off')

plt.show()

In [ ]:
img, pred_mask, main = model_predict('images/train_14.png')

# 결과 표시
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('SafeZone From Fire :: Input')
ax1.axis('off')

ax2.imshow(pred_mask, cmap='gray')
ax2.set_title('SafeZone From Fire :: Mask')
ax2.axis('off')

ax3.imshow(main)
ax3.set_title('SafeZone From Fire :: Contours')
ax3.axis('off')

plt.show()

In [ ]:
img, pred_mask, main = model_predict('images/train_21210.png')

# 결과 표시
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('SafeZone From Fire :: Input')
ax1.axis('off')

ax2.imshow(pred_mask, cmap='gray')
ax2.set_title('SafeZone From Fire :: Mask')
ax2.axis('off')

ax3.imshow(main)
ax3.set_title('SafeZone From Fire :: Contours')
ax3.axis('off')

plt.show()